In [280]:
from quantopian.pipeline.data import Fundamentals  
from quantopian.interactive.data.psychsignal import aggregated_twitter_withretweets_stocktwits
from quantopian.pipeline.data.sentdex import sentiment
from quantopian.pipeline.data import factset
from quantopian.pipeline.data.psychsignal import stocktwits
from sklearn.preprocessing import StandardScaler

In [95]:
from sklearn.linear_model import LinearRegression

In [96]:
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline import factors, filters, classifiers, Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.filters import QTradableStocksUS

In [138]:
def price_pipeline():
    universe = QTradableStocksUS() & (USEquityPricing.open.latest > 1)

    market_cap = Fundamentals.market_cap.latest  
    market_filter = market_cap.percentile_between(90,100,
                                                    mask = universe)
    
    
    return Pipeline(
            columns = {
            'market_cap' : market_cap,
            },
            screen = market_filter,
            )
    

In [139]:
result = run_pipeline(price_pipeline(), start_date = '05-01-2019', end_date = '05-01-2019')

In [140]:
len(result)

214

In [157]:
# consider today as 05-01-2019
df = get_pricing('AAPL',
               end_date = '05-01-2019',
               start_date = '05-01-2016',
               fields = ['open_price', 'close_price'])

In [258]:
# lin reg or 36 month
X = df.index.values
y = df['open_price']
lr = LinearRegression()
lr.fit(X.reshape(-1,1),y)

In [295]:
#  drop last month and find 12 month
new_df = df[21:272]
X_2 = new_df.index.values.astype('float')
y_2 = new_df['open_price']
preds = lr.predict(X_2.reshape(-1,1))

In [291]:
# find sum zscore
new_df['diff'] = y_2 - preds
new_df['zscore'] = (new_df['diff'] - new_df['diff'].mean())/new_df['diff'].std(ddof=0)
sum(new_df['zscore'])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [300]:
MY_STOCKS = symbols(['SPY', 'EFA', 'BND', 'VNQ', 'GSG'])

In [302]:
def linreg_pipline():
    universe = Filters.StaticAssets(MY_STOCKS)
    
    
    factor = []
    for stock in MY_STOCKS:
        df = get_pricing(stock,
               end_date = '05-01-2019',
               start_date = '05-01-2016',
               fields = ['open_price', 'close_price'])
        
        X = df.index.values
        y = df['open_price']
        lr = LinearRegression()
        lr.fit(X.reshape(-1,1),y)
        
        new_df = df[21:272]
        X_2 = new_df.index.values.astype('float')
        y_2 = new_df['open_price']
        preds = lr.predict(X_2.reshape(-1,1))
        
        
        new_df['diff'] = y_2 - preds
        new_df['zscore'] = (new_df['diff'] - new_df['diff'].mean())/new_df['diff'].std(ddof=0)
        factor.append(sum(new_df['zscore']))
        
    pip = Pipelin(
    columns = {
        'Z_scrore' : factor
    },
    screen = universe)
    
    return pipe

In [ ]:
result = run_pipeline(linreg_pipelin(), start_date)

In [197]:
import pandas as pd
import numpy as np
from statsmodels import regression
import statsmodels.api as sm
import matplotlib.pyplot as plt
import math

def linreg(X,Y):
    # Running the linear regression
    X = sm.add_constant(X)
    
    a = model.params[0]
    b = model.params[1]
    X = X[:, 1]

    # Return summary of the regression and plot results
    X2 = np.linspace(X.min(), X.max(), 100)
    Y_hat = X2 * b + a
    plt.scatter(X, Y, alpha=0.3) # Plot the raw data
    plt.plot(X2, Y_hat, 'r', alpha=0.9);  # Add the regression line, colored in red
    plt.xlabel('X Value')
    plt.ylabel('Y Value')
    return model.summary()